# Eigen Decomposition and SVD Applications

https://github.com/mikexcohen/LinAlg4DataScience/blob/main/LA4DS_ch15.ipynb

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec # for the subplots

import pandas as pd
import seaborn as sns